<a href="https://colab.research.google.com/github/arogya-gyawali/brainscan_AI/blob/main/notebooks/data_prep/03_convert_healthy_IXI_nii_to_png.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Convert Healthy IXI .nii Files to PNG Slices

> ⚠️ This notebook processes MRI scans from the **IXI dataset** (healthy brain samples in `.nii` format).

## 📦 Input
- Folder: `/content/drive/MyDrive/BrainScanAI/IXI-T1`
- Files: T1-weighted MRI `.nii` volumes (3D)

## 🎯 Output
- Folder: `/content/drive/MyDrive/BrainScanAI/BrainScanAI_final_output/no_tumor_IXI_png`
- For each volume: 3 mid-slices (1 before, center, 1 after)
- Each slice saved as **grayscale `.png`**
- Metadata saved as `no_tumor_metadata.csv`

## 🧪 Purpose
This data represents **non-tumorous brain scans** and is useful for:
- Training/validating classifiers to distinguish healthy vs tumorous MRIs
- Augmenting tumor datasets with healthy control samples

## 📍Note
You may need to mount Google Drive and adjust paths to match your setup.


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
import os
import nibabel as nib
import numpy as np
from PIL import Image
import pandas as pd

# === INPUT/OUTPUT PATHS ===
nii_input_folder = "/content/drive/MyDrive/BrainScanAI/IXI-T1"  # folder with .nii files
png_output_folder = "/content/drive/MyDrive/BrainScanAI/BrainScanAI_final_output/no_tumor_IXI_png"  # where PNGs will go
os.makedirs(png_output_folder, exist_ok=True)

# === CONFIGURATION ===
num_slices_each_side = 1  # total slices = 5 (center + 2 before + 2 after)

# === METADATA STORAGE ===
metadata = []

# === LIST AND PROCESS .nii FILES ===
nii_files = sorted([f for f in os.listdir(nii_input_folder) if f.endswith(".nii")])

for i, file_name in enumerate(nii_files):
    file_path = os.path.join(nii_input_folder, file_name)

    try:
        # Load the .nii file
        nii_image = nib.load(file_path)
        image_data = nii_image.get_fdata()

        z_dim = image_data.shape[2]
        mid_slice = z_dim // 2

        for offset in range(-num_slices_each_side, num_slices_each_side + 1):
            slice_index = mid_slice + offset
            if slice_index < 0 or slice_index >= z_dim:
                continue

            slice_img = image_data[:, :, slice_index]

            # Normalize to 0–255
            norm_img = 255 * (slice_img - np.min(slice_img)) / np.ptp(slice_img)
            norm_img = norm_img.astype(np.uint8)

            base_name = os.path.splitext(file_name)[0]
            output_filename = f"{base_name}_slice{slice_index}.png"
            output_path = os.path.join(png_output_folder, output_filename)

            # Save as PNG
            Image.fromarray(norm_img).save(output_path)

            # Add to metadata
            metadata.append({
                "file": output_filename,
                "label": 0,  # Healthy brain
                "source": "IXI",
                "slice_index": slice_index
            })

        if i % 25 == 0:
            print(f"✅ Processed {i}/{len(nii_files)}: {file_name}")

    except Exception as e:
        print(f"❌ Error processing {file_name}: {e}")

# === SAVE METADATA ===
df = pd.DataFrame(metadata)
csv_path = os.path.join(png_output_folder, "no_tumor_metadata.csv")
df.to_csv(csv_path, index=False)

print(f"🏁 Finished. Saved {len(metadata)} slices and metadata to:\n→ {csv_path}")


✅ Processed 0/581: IXI002-Guys-0828-T1.nii
✅ Processed 25/581: IXI038-Guys-0729-T1.nii
✅ Processed 50/581: IXI064-Guys-0743-T1.nii
✅ Processed 75/581: IXI090-Guys-0800-T1.nii
✅ Processed 100/581: IXI115-Guys-0738-T1.nii
✅ Processed 125/581: IXI143-Guys-0785-T1.nii
✅ Processed 150/581: IXI173-HH-1590-T1.nii
✅ Processed 175/581: IXI200-Guys-0812-T1.nii
✅ Processed 200/581: IXI228-Guys-0822-T1.nii
✅ Processed 225/581: IXI256-HH-1723-T1.nii
✅ Processed 250/581: IXI286-Guys-0859-T1.nii
✅ Processed 275/581: IXI312-Guys-0887-T1.nii
✅ Processed 300/581: IXI338-HH-1971-T1.nii
✅ Processed 325/581: IXI370-Guys-0921-T1.nii
✅ Processed 350/581: IXI396-HH-2115-T1.nii
✅ Processed 375/581: IXI422-Guys-1071-T1.nii
✅ Processed 400/581: IXI447-Guys-0979-T1.nii
✅ Processed 425/581: IXI475-IOP-1139-T1.nii
✅ Processed 450/581: IXI500-Guys-1017-T1.nii
✅ Processed 475/581: IXI531-Guys-1057-T1.nii
✅ Processed 500/581: IXI559-HH-2394-T1.nii
✅ Processed 525/581: IXI589-Guys-1080-T1.nii
✅ Processed 550/581: IXI61